## Requirements
It is expected that simulator can perform following:

- initialize state
- read program, and for each gate:
    - calculate matrix operator
    - apply operator (modify state)

- perform multi-shot measurement of all qubits using weighted random technique

It is up to you how you will organize code, but this is our suggestion:

### Input format (quantum program)

It is enough if simulator takes program in following format:

```
[
  { \"unitary\": [[0.70710678, 0.70710678], [0.70710678, -0.70710678]], \"target\": [0] },
  { \"unitary\": [ [1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 0, 1], [0, 0, 1, 0] ], \"target\": [0, 1] }
  ...
]
```

Or, you can define unitaries in the code, and accept program with gate names instead:

```
[
  { \"gate\": \"h\", \"target\": [0] },
  { \"gate\": \"cx\", \"target\": [0, 1] }
  ...
]
```

In [29]:
import numpy as np
import unittest
from numpy.random import multinomial
import math
import json
import functools

# Get ground state [DONE]

In [ ]:
def get_ground_state(num_qubits):
    # return vector of size 2**num_qubits with all zeroes except first element which is 1

#     assert num_qubits > 0
    
    ground_state = a = np.zeros(2**num_qubits)
    ground_state[0] = 1
    return ground_state

In [ ]:
class TestGetGroundState(unittest.TestCase):

#     def test_invalid_input(self):
#         self.assertRaises(get_ground_state(0), AssertionError)

    def test_single_qubit(self):
        self.assertSequenceEqual(
            list(get_ground_state(1)),
            list(np.array([1, 0]))
        )

    def test_multi_qubit(self):
        self.assertSequenceEqual(
            list(get_ground_state(3)),
            list(np.array([1, 0, 0, 0, 0 ,0, 0, 0]))
        )

suite = unittest.TestLoader().loadTestsFromTestCase(TestGetGroundState)
unittest.TextTestRunner().run(suite)

# Get Operator [not started]

In [4]:
# Define 2x2 Identity
I = np.identity(2)

# Define X gate
X = np.array([
[0, 1],
[1, 0]
])

# Define Y gate
Y = np.array([
[0, 0-1j],
[0+1j, 0]
])

# Define Z gate
Z = np.array([
[1, 0],
[0, -1]
])


# Define projection operator |0><0|
P0x0 = np.array([
[1, 0],
[0, 0]
])

# Define projection operator |1><1|
P1x1 = np.array([
[0, 0],
[0, 1]
])

KNOWN_GATES = [I, X, Y, Z, P0x0, P1x1]

In [57]:
def get_operator(total_qubits, gate_unitary, target_qubits):
    # return unitary operator of size 2**n x 2**n for given gate and target qubits
    
#     def product(position, product, gate_unitary, target_qubits, total_qubits):
#         if position = total_qubits:
#             return product
        
#         if position = target_qubits[0]:
#             return product(position+1, )
#         return gate_unitary
    
#     np.kron(I, gate_unitary)
    ###########################
    
    def multiply_gates(gate_unitary, target_qubit, projector=None, control_qubit=None):
        gates = [I] * total_qubits
        gates[target_qubit] = gate_unitary
        if projector is not None and control_qubit is not None:
            gates[control_qubit] = projector
        return functools.reduce(lambda a,b : np.kron(a, b), gates)
    
    if len(target_qubits) == 1:
#         gates = [I] * total_qubits
#         target_qubit = target_qubits[0] 
#         gates[target_qubit] = gate_unitary
#         result = functools.reduce(lambda a,b : np.kron(a, b), gates)
#         print('gate:\n', np.array2string(result))
        result = multiply_gates(gate_unitary, target_qubits[0])
        print('gate:\n', np.array2string(result))
        return result
    else:
        result\
        = multiply_gates(gate_unitary, target_qubits[1], P0x0, target_qubits[0])\
        + multiply_gates(gate_unitary, target_qubits[1], P1x1, target_qubits[0])
        print('controlled-gate:\n', np.array2string(result))
        return result
    
    return gate_unitary

In [56]:
class TestGetOperator(unittest.TestCase):

    def test_n_is_1_X_on_0(self):
        np.testing.assert_array_equal(
            get_operator(1, X, [0]),
            np.array([
                [0,1],
                [1,0]
            ])
        )

    def test_n_is_2_X_on_0(self):
        np.testing.assert_array_equal(
            get_operator(2, X, [0]),
            np.array([
                [0,0,1,0],
                [0,0,0,1],
                [1,0,0,0],
                [0,1,0,0]
            ])
        )
    
    def test_n_is_2_X_on_1(self):
        np.testing.assert_array_equal(
            get_operator(2, X, [1]),
            np.array([
                [0,1,0,0],
                [1,0,0,0],
                [0,0,0,1],
                [0,0,1,0]
            ])
        )

    def test_n_is_3_X_on_2(self):
        np.testing.assert_array_equal(
            get_operator(3, X, [2]),
            np.array([
                [0, 1, 0, 0, 0, 0, 0, 0,],
                [1, 0, 0, 0, 0, 0, 0, 0,],
                [0, 0, 0, 1, 0, 0, 0, 0,],
                [0, 0, 1, 0, 0, 0, 0, 0,],
                [0, 0, 0, 0, 0, 1, 0, 0,],
                [0, 0, 0, 0, 1, 0, 0, 0,],
                [0, 0, 0, 0, 0, 0, 0, 1,],
                [0, 0, 0, 0, 0, 0, 1, 0,]
            ])
        )

suite = unittest.TestLoader().loadTestsFromTestCase(TestGetOperator)
unittest.TextTestRunner().run(suite)

....

gate:
 [[0 1]
 [1 0]]
gate:
 [[0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]]
gate:
 [[0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]]
gate:
 [[0. 1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1. 0.]]



----------------------------------------------------------------------
Ran 4 tests in 0.006s

OK


<unittest.runner.TextTestResult run=4 errors=0 failures=0>

# Run Program [not started]


In order to implement simulator, it is best if you have function which returns operator for any unitary targeting any qubit(s) for any circuit size, something like:

```get_operator(total_qubits, gate_unitary, target_qubits)```

But this is not trivial so it is enough if you can implement it for any 1-qubit gates and CNOT only.

If you are still enthusiastic and you wish to implement universal operator function then please refer to:

- [qosf-simulator-task-additional-info.pdf|https://github.com/quantastica/qosf-mentorship/blob/master/qosf-simulator-task-additional-info.pdf]

- Book Nielsen, Michael A.; Chuang, Isaac (2000). Quantum Computation and Quantum Information, 10th Anniversary Edition, Section 8.2.3, Operator-Sum Representation




In [ ]:
def run_program(initial_state, program):
    # read program, and for each gate:
    #   - calculate matrix operator
    #   - multiply state with operator
    # return final state
    return

# Measure All [DONE; remove comments]

In [ ]:
def measure_all(state_vector):
    # choose element from state_vector using weighted random and return it's index
    def get_probability(amplitude):
        return np.abs(amplitude)**2
    
    probabilities = np.fromiter((get_probability(amplitude) for amplitude in state_vector), state_vector.dtype) 
#     print('prob', probabilities)
    
    result_array = multinomial(1, probabilities)
#     print('results', result_array)    
    indices = np.where(result_array == 1)
#     print('indices: ', indices)    
    index = indices[0][0]
#     print('index: ', index)
    
    bin_index = bin(index)[2:].zfill(math.frexp((len(state_vector)))[1] - 1)
#     print('bin index: ', bin_index)
    
    return bin_index

In [ ]:
class TestMeasureAll(unittest.TestCase):

    def test_measure_bell_00(self):
        actualResult = measure_all(np.array([0.70710678, 0, 0, -0.70710678]))
        self.assertTrue(
           (actualResult == '00') or (actualResult == '11')
        )

suite = unittest.TestLoader().loadTestsFromTestCase(TestMeasureAll)
unittest.TextTestRunner().run(suite)

# Get Counts [remove comments + fix test]

In [ ]:
def get_counts(state_vector, num_shots):
    # simply execute measure_all in a loop num_shots times and
    # return object with statistics in following form:
    #   {
    #      element_index: number_of_ocurrences,
    #      element_index: number_of_ocurrences,
    #      element_index: number_of_ocurrences,
    #      ...
    #   }
    # (only for elements which occoured - returned from measure_all)

    counts = {} 
    # or listofzeros = [0] * len(state_vector)
    for x in range(num_shots):
        index = measure_all(state_vector)
        counts[index] = counts.get(index, 0) + 1
        # counts[measure_all(state_vector)] += 1 
        
    sorted_counts = json.dumps(counts, sort_keys=True, indent=4)
    print(sorted_counts)

    # time considerarions of dic vs list
    # list needs extra pass to covert index to binary
    # dic can take binary string from measure all, but does this affect performance of get_counts
    # dic->json goves output in required format
    
    return sorted_counts

In [ ]:
class TestGetCounts(unittest.TestCase):

    def test_run_1000_shots(self):
        self.assertEqual(
            get_counts(np.array([0.70710678, 0, 0, -0.70710678]), 1000),
            0
        )

suite = unittest.TestLoader().loadTestsFromTestCase(TestGetCounts)
unittest.TextTestRunner().run(suite)

# Test execution

In [ ]:
unittest.main(argv=[''], verbosity=0, exit=False)